In [3]:
!pip install --quiet transformers accelerate bitsandbytes peft datasets wandb

In [4]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


In [5]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset

In [6]:
dataset = load_dataset("marmikpandya/mental-health")

print("Columns:", dataset["train"].column_names)
print("Train size:", len(dataset["train"]))

dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

print("Train examples:", len(train_dataset))
print("Eval examples:", len(eval_dataset))

data.jsonl:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13358 [00:00<?, ? examples/s]

Columns: ['instruction', 'output', 'input']
Train size: 13358
Train examples: 12022
Eval examples: 1336


In [7]:
model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

print("Model loaded successfully.")

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded successfully.


In [8]:
# Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
)

# Wrap the model with LoRA
model = get_peft_model(model, peft_config)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"Trainable parameters: {trainable_params}")
print(f"Total parameters: {total_params}")
print(f"Trainable parameters percentage: {100 * trainable_params / total_params:.2f}%")

Trainable parameters: 9175040
Total parameters: 1530567680
Trainable parameters percentage: 0.60%


In [9]:
def preprocess_function(examples):
    instructions = examples["instruction"]
    inputs_ = examples["input"]
    outputs = examples["output"]
    
    text_list = []
    for instr, inp, outp in zip(instructions, inputs_, outputs):
        # Construct a basic prompt
        prompt = (
            f"Instruction: {instr.strip()}\n"
            f"Input: {inp.strip()}\n"
            f"Response: {outp.strip()}"
        )
        text_list.append(prompt)
    
    # Tokenize 
    tokenized = tokenizer(
        text_list,
        max_length=512,       
        truncation=True,
        padding="max_length"
    )
    return tokenized

train_dataset = train_dataset.map(
    preprocess_function, 
    batched=True,
    remove_columns=["instruction", "input", "output"]
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["instruction", "input", "output"]
)

print("Sample preprocessed example:", train_dataset[0])

Map:   0%|          | 0/12022 [00:00<?, ? examples/s]

Map:   0%|          | 0/1336 [00:00<?, ? examples/s]

Sample preprocessed example: {'input_ids': [6310, 2762, 25, 1002, 345, 389, 257, 11971, 23540, 11, 3387, 2148, 428, 5827, 351, 257, 7613, 2882, 284, 511, 2328, 13, 198, 20560, 25, 314, 1053, 587, 1719, 5876, 11029, 290, 314, 892, 340, 338, 780, 286, 616, 1693, 13, 198, 31077, 25, 36957, 422, 670, 460, 4753, 2928, 534, 3993, 13, 3914, 338, 7301, 2842, 284, 6687, 534, 5503, 290, 2987, 534, 3993, 33306, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5025

In [10]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="phi-2_mental_health",
    overwrite_output_dir=True,
    num_train_epochs=2,  
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy='steps',      
    eval_steps=500,                  # Evaluate every 500 steps
    save_strategy='steps',
    save_steps=500,                   
    learning_rate=5e-5,
    warmup_steps=500,
    gradient_accumulation_steps=4, 
    logging_steps=500,
    fp16=True,          
    report_to="none"     
)

print("Training args prepared.")

Training args prepared.


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss,Validation Loss
500,1.333400,1.101011
1000,1.159600,1.065751
1500,1.109400,1.046436
2000,1.094800,1.033542
2500,1.079100,1.026899
3000,1.050400,1.022064


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


TrainOutput(global_step=3004, training_loss=1.1378107145527867, metrics={'train_runtime': 19564.7664, 'train_samples_per_second': 1.229, 'train_steps_per_second': 0.154, 'total_flos': 1.9621458217009152e+17, 'train_loss': 1.1378107145527867, 'epoch': 1.999001829978373})

In [12]:
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

trainer.save_model("phi-2_mental_health")
tokenizer.save_pretrained("phi-2_mental_health")
print("Model and tokenizer saved.")

Evaluation results: {'eval_loss': 1.0220448970794678, 'eval_runtime': 465.9353, 'eval_samples_per_second': 2.867, 'eval_steps_per_second': 1.434, 'epoch': 1.999001829978373}
Model and tokenizer saved.


In [2]:
from transformers import pipeline

# Initialize the pipeline for text generation with LoRA fine-tuned model
generator = pipeline("text-generation",
    model="/kaggle/working/phi-2_mental_health",
    tokenizer="/kaggle/working/phi-2_mental_health",
    device=1
)

def generate_response(instruction, input_text, max_new_tokens=256):
    prompt = (
        f"Instruction: {instruction}\n"
        f"Input: {input_text}\n"
        f"Response:"
    )
    response = generator(prompt, max_length=256, num_return_sequences=1, do_sample=True, temperature=0.7, top_p=0.9)
    return response[0]["generated_text"]

# Test the model
test_instruction = "You are a mental health assistant designed to offer support and guidance to individuals who are navigating mental health challenges. Your goal is to provide empathetic, non-judgmental, and helpful responses to users, guiding them toward healthier coping mechanisms and encouraging them to seek professional help when needed."
test_input = "I am not feeling well now a days as the job market is bad and not getting job?"
answer = generate_response(test_instruction, test_input)
print("Generated response:\n", answer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Generated response:
 Instruction: You are a mental health assistant designed to offer support and guidance to individuals who are navigating mental health challenges. Your goal is to provide empathetic, non-judgmental, and helpful responses to users, guiding them toward healthier coping mechanisms and encouraging them to seek professional help when needed.
Input: I am not feeling well now a days as the job market is bad and not getting job?
Response: I'm sorry to hear that you're going through a difficult time. Job market conditions can be tough, but remember that you're not alone in this. It's important to take care of yourself during this challenging period. Consider reaching out to local job placement services or career counseling centers for additional support. In the meantime, make sure to take time for self-care activities that help you relax and recharge.

